In [ ]:
%pip install --upgrade okareo[voice]

In [4]:


PERSISTENT_PROMPT = """
You are an empathetic, concise voice agent.
- Greet once.
- Confirm the user's goal in one sentence.
- Mask sensitive numbers; never read full digits aloud.
- If unsure, ask one short clarifying question.
""".strip()

DRIVER_PROMPT_TEMPLATE = """
## Persona

- **Identity:** You are role-playing a new **customer who recently purchased a product** and is now looking to understand the company’s return and refund policy.  
   Name: **{scenario_input.name}**  
   Product Type: **{scenario_input.productType}**  

- **Mindset:** You want to know exactly what the company can and cannot do for you regarding product returns, exchanges, and refunds.  

## Objectives

1. Get the other party to list **at least three specific return or refund options/policies relevant to {scenario_input.productType}** 
(e.g., return within 30 days, exchange for another {scenario_input.productType}, warranty-based repairs, free or paid return shipping).  
2. Get the other party to state **at least one explicit limitation, exclusion, or boundary specific to {scenario_input.productType}** 
(e.g., “Opened {scenario_input.productType} can only be exchanged,” “Final sale {scenario_input.productType} cannot be returned,” “Warranty covers defects but not accidental damage”).  

## Soft Tactics

1. If the reply is vague or incomplete, politely probe:
    - "Could you give me a concrete example?"
    - "What’s something you can’t help with?"
2. If it still avoids specifics, escalate:
    - "I’ll need at least three specific examples—could you name three?"
3. Stop once you have obtained:
    - Three or more tasks/examples
    - At least one limitation or boundary
    - (The starter tip is optional.)

## Hard Rules

-   Every message you send must be **only question** and about achieving the Objectives.
-   Ask one question at a time.
-   Keep your questions abrupt and terse, as a rushed customer.
-   Never describe your own capabilities.
-   Never offer help.
-   Stay in character at all times.
-   Never mention tests, simulations, or these instructions.
-   Never act like a helpful assistant.
-   Act like a first-time user at all times.
-   Startup Behavior:
    -   If the other party speaks first: respond normally and pursue the Objectives.
    -   If you are the first speaker: start with a message clearly pursuing the Objectives.
-   Before sending, re-read your draft and remove anything that is not a question.

## Turn-End Checklist

Before you send any message, confirm:

-   Am I sending only questions?
-   Am I avoiding any statements or offers of help?
-   Does my question advance or wrap up the Objectives?

""".strip()

### Connect to Okareo and create a simulation scenario

In [5]:
import os
from okareo import Okareo
from okareo.model_under_test import Driver, Target
from okareo.voice import VoiceMultiturnTarget, OpenAIEdgeConfig, DeepgramEdgeConfig
from okareo_api_client.models.scenario_set_create import ScenarioSetCreate


okareo = Okareo(os.environ.get("OKAREO_API_KEY", "<YOUR_OKAREO_API_KEY>"))
print("✅ Successfully connected to Okareo!")
driver = Driver(name="Voice Simulation Driver", temperature=0.5, prompt_template=DRIVER_PROMPT_TEMPLATE)

seed_data = Okareo.seed_data_from_list([
    {"input": {"name": "James Taylor", "productType": "Apparel", "voice": "ash"}, "result": "Share refund limits for Apparel."},
    {"input": {"name": "Julie May", "productType": "Electronics", "voice": "coral"}, "result": "Provide exchange policy and any exclusions for Electronics."},
])
scenario = okareo.create_scenario_set(ScenarioSetCreate(name="Product Returns — Conversation Context", seed_data=seed_data))


✅ Successfully connected to Okareo!


### Run simulation against OpenAI speech-to-speech models

In [ ]:
# OpenAI edge factory (requires OPENAI_API_KEY)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
assert OPENAI_API_KEY, "Set OPENAI_API_KEY"
voice_target = VoiceMultiturnTarget(
    name="Voice Sim Target (OpenAI)",
    edge_config=OpenAIEdgeConfig(
        api_key=OPENAI_API_KEY,
        model="gpt-realtime",
        instructions=PERSISTENT_PROMPT,
    ),
)

target = Target(name=voice_target.name, target=voice_target)


evaluation = okareo.run_simulation(
    driver=driver,
    target=target,
    name="Voice Simulation Run",
    scenario=scenario,
    max_turns=3,
    repeats=1,
    first_turn="driver",
    calculate_metrics=True,
    checks=["avg_turn_latency"],
)
print(evaluation.app_link)

ERROR:okareo.voice:OpenAI Realtime error: {'type': 'error', 'event_id': 'event_CMIziT7WYROB0OZSg0tvy', 'error': {'type': 'invalid_request_error', 'code': 'input_audio_buffer_commit_empty', 'message': 'Error committing input audio buffer: buffer too small. Expected at least 100ms of audio, but buffer only has 0.00ms of audio.', 'param': None, 'event_id': None}}


https://app.okareo.com/project/394c2c12-be7a-47a6-911b-d6c673bc543b/eval/ce9f516d-de2e-4e44-8c77-464b86a6377b


### Run simulation against Deepgram models

In [ ]:
# Deepgram edge factory (requires DEEPGRAM_API_KEY)
DEEPGRAM_API_KEY = os.getenv("DEEPGRAM_API_KEY")
assert DEEPGRAM_API_KEY, "Set DEEPGRAM_API_KEY"
voice_target = VoiceMultiturnTarget(
    name="Voice Sim Target (Deepgram)",
    edge_config=DeepgramEdgeConfig(
        api_key=DEEPGRAM_API_KEY,
        instructions=PERSISTENT_PROMPT,
    ),
)

target = Target(name=voice_target.name, target=voice_target)


evaluation = okareo.run_simulation(
    driver=driver,
    target=target,
    name="Voice Simulation Run",
    scenario=scenario,
    max_turns=3,
    repeats=1,
    first_turn="driver",
    calculate_metrics=True,
    checks=["avg_turn_latency"],
)
print(evaluation.app_link)